In [1]:
import numpy as np
import scipy
import math

## Estimating the likelihood of a site given a tree

To make it a bit more explicit, I’m explaining in more details here the step 2 of the project.

When you create your tree from the three files, the only nodes having information about the data are the tips (terminal nodes). This is because we do have data for them in the form of sequences in the alignment. The internal nodes are not observed and we do not know the sequence that they could hold. The main goal of the algorithm is to take into account the uncertainty in the values of the internal nodes to get an estimate of the tree likelihood.

The algorithm to calculate the likelihoods will start from the data at each tip and we will assume that each site in the msa is independent. We can thus calculate the likelihood of a single site at a time. The full likelihood of a msa will be the product of the likelihoods at each site.

As said before, at the internal nodes, we do not observe any data. There is thus an uncertainty on whether the node has the nucleotide $A$, $C$, $G$ or $T$ at site $s$. To take this into account, we need to calculate a vector of likelihood at each node for the site $s$. The vector has 4 elements, which are the likelihoods to oberved $A$, $C$, $G$ or $T$ at that node. We call these vectors conditional likelihoods, because for internal nodes, the values are conditional on the data that is present in the two nodes directly descending from the one we are looking at. At the terminal nodes, we do observe the data. For example, say that sequence 1 has a $A$ for the site $s$. We will encode this into a vector format too, to make it compatible with what we use for internal nodes. Therefore, the data $A$ will be coded as `[1,0,0,0]`. We are certain to see $A$ at this site, because it is in the data itself. We usually keep the order $A$, $C$, $G$, $T$ for the four elements of each of these vectors.

Imagine that we have a simple tree with two terminal nodes and one internal node. Let's try to estimate the likelihood on this simplified example to make it clear. Assume that the `species1` has a $A$ and `species2` has a $C$. The data is the following (i.e. a list in Python):

In [2]:
species1 = [1,0,0,0]
species2 = [0,1,0,0]

Assume next that `species1` is connected to their common ancestor by a branch of length $t_1=0.1$, while `species2` is connected by a branch of length $t_2=0.15$. These branches express the amount of evolution that occurred between the ancestor and the two species. They are in units of expected number of substitutions per site.

We would like to estimate which nucleotide is most probable in the ancestor given this data. To do so, we need another element, which is a model of evolution, to tell us the probabilities of changing into a $A$ in `species1` and a $C$ in `species2` given an unknown nucleotide $X$ in the ancestor. This is done by setting a transition matrix $Q$ of size $4 \times 4$, with rows representing the starting nucleotide and columns the ending nucleotide. We need to combine this model of evolution with the amount of evolution between two nodes (express by the branch lengths) to get a probability matrix $P$ of having a certain nucleotide at the end of the branch if we have a certain nucleotide at the beginning of it. This is done by exponentiating the product of $Q$ and the branch length.

In our project, we will assume that $Q$ is fixed and you will create it using the `numpy.array()` function:

In [3]:
u=0.3
Q = np.array([[-3*u,u,u,u],[u,-3*u,u,u],[u,u,-3*u,u],[u,u,u,-3*u]])
print(f"Q={Q}")

Q=[[-0.9  0.3  0.3  0.3]
 [ 0.3 -0.9  0.3  0.3]
 [ 0.3  0.3 -0.9  0.3]
 [ 0.3  0.3  0.3 -0.9]]


The values in the $Q$ matrix represent the rate of change from one nucleotide to the other (rate of substitution, here set to 1, times the frequency of arriving at a nucleotide, here set to 0.25). The matrix $Q$ is then scaled by the total amount of change to have the total rate of change being 1 (not directly obvious from $Q$, but trust me). The scaling is important because the $Q$ matrix appears as a product with the branch length to obtain the probabilities of change. Scaling $Q$ means that the branch length is the parameter reflecting the amount of change. The model presented in the $Q$ matrix here is called the Jukes-Cantor model (from Jukes and Cantor 1969).

For the two branches $t_1$ and $t_2$, we estimate the probability of change along the branch by exponentiating the matrix $Q$ times the branch length. This gives the probability matrix $P$. In Python, this is done as follows

In [7]:
#probability of change along the branch given the branch length (exponentiating matrix Q times the branch length)
t1 = 0.1
t2 = 0.15
P1 = scipy.linalg.expm(Q * t1) #t1=0.1
P2 = scipy.linalg.expm(Q * t2) #t2=0.15
print(f"P1={P1}")
print(f"P2={P2}")

P1=[[0.91519033 0.02826989 0.02826989 0.02826989]
 [0.02826989 0.91519033 0.02826989 0.02826989]
 [0.02826989 0.02826989 0.91519033 0.02826989]
 [0.02826989 0.02826989 0.02826989 0.91519033]]
P2=[[0.87645266 0.04118245 0.04118245 0.04118245]
 [0.04118245 0.87645266 0.04118245 0.04118245]
 [0.04118245 0.04118245 0.87645266 0.04118245]
 [0.04118245 0.04118245 0.04118245 0.87645266]]


We will get the probability of starting from $A$ in `species1` and ending with either $A$, $C$, $G$ or $T$ in the ancestor by multiplying $P_1$ with the data vector `[1,0,0,0]`. We do the same for `species2` with $P_2$ and the data vector `[0,1,0,0]`. In Python:

In [12]:
vec1 = np.matmul(P1, [1,0,0,0]) #now we have to get probabilities of getting A based on the branch length and Q matrix
vec2 = np.matmul(P2, [0,1,0,0]) #probabilities of getting C based on its branch length
print(f"vec1={vec1}")
print(f"vec2={vec2}")

vec1=[0.91519033 0.02826989 0.02826989 0.02826989]
vec2=[0.04118245 0.87645266 0.04118245 0.04118245]


We have now two realizations of an evolutionary path from an unknown ancestor to the two species with data $A$ and $C$. The two species have the ancestor in common because they are connected by the tree. These two realizations should then be combined to obtain the joint probability of evolving $A$ and $C$ along the branches $t_1$ and $t_2$, respectively, from the shared ancestor. This is done by multiplying element-wise the two vectors obtained to get the conditional likelihood vector for the ancestor. In Python:

In [14]:
vec_anc = [vec1[i] * vec2[i] for i in range(4)]
print(f"vec_anc={vec_anc}")

vec_anc=[np.float64(0.03768977729344043), np.float64(0.02477722096696906), np.float64(0.0011642232845805202), np.float64(0.0011642232845805202)]


This result tells us that the probability that the ancestor has an $A=0.0251$, a $C=0.0166$, etc... These numbers do not sum up to one, because it is the conditional probability of observing the nucleotides given the branch lengths and the data in the two species. We could therefore have many more ways to get the probabilities that we do not cover with this specific tree, branch lengths and model. However, it makes sense that the probability of $A$ is higher than $C$ because the branch length $t_1$ is smaller than $t_2$. There is thus less evolution leading to the $A$ than to the $C$, which implies that the ancestor is slighly more probable to have a $A$. Nice, no? Nucleotides $G$ and $T$ are also highly unlikely, which makes again sense as we do not observe them in the data.

The log-likelihood of the site $s$ (i.e. probability of the data $A$, $C$, given the tree, branch length and model) will then be the logarithm of the sum of the multiplication of the conditional vector with the probabilities of having $A$, $C$, $G$ and $T$, which, for this model is $1/4$ each. In Python:

In [10]:
npmm = np.matmul(vec_anc, [0.25,0.25,0.25,0.25])
print(f"npmm={npmm}")
print(f"log-likelihood is {math.log(npmm)}")

npmm=0.016198861207392633
log-likelihood is -4.122814335054628


Likelihoods are tiny values and when you have many sites $s$, multiplying them together leads to underflow during the computations. We usually transform the likelihoods into log-likelihoods. Any products become then an addition (i.e. $log(\prod_{i=0}^n l_i) = \sum_{i=0}^n log(l_i)$).

Our algorithm therefore returns at the end the log-likelihood of the site $s$ given the tree topology (trivial here as we have only two species), the vector of branch lengths and the model of evolution $Q$.

### Adding another species

If our tree had three species, with the third one sharing a common ancestor with the ancestor of `species1` and `species2` that we just estimated, we would keep going with our algorithm one step further. Imagine that `species3` had the nucleotide $C$ and a branch length $t_3 = 0.12$, that the branch connecting the ancestor of `species1` and `species2` (let's call it `anc12`) and the common ancestor of all (ie the root) was $t_4 = 0.04$. The data of `anc12` would not be known, but we could use the conditional likelihood that we estimated for it (and stored in the list `vec_anc`). Thus, in Python, the likelihood of this slightly larger tree would be:

In [11]:
t3, t4 = 0.12, 0.04
P3 = scipy.linalg.expm(Q * t3) #t3=0.12
P4 = scipy.linalg.expm(Q * t4) #t4=0.04
vec3 = np.matmul(P3, [0,1,0,0]) #species3 has a C
vec4 = np.matmul(P4, vec_anc) #the descendant here is anc12. We don't have data, but our best guess is the conditional likelihood estimated before, ie vec_anc
vec_root = [vec3[i] * vec4[i] for i in range(4)]
npmm = np.matmul(vec_root, [0.25,0.25,0.25,0.25])
print(math.log(npmm))

-5.146513419401049


This is why you need a data structure to store the tree that allows you to navigate through the different nodes to estimate the conditional likelihoods for the internal nodes. You should start from the root, check if its two descendants have been estimated or not. If yes, you can estimate the likelihood of the site directly. If not, you need to set the conditional likelihoods of the descendants recursively...